***GENERATED CODE FOR losregression PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
from pyspark.sql.functions import dayofmonth, month, year, col
import json
from pyspark.ml.feature import Binarizer
from pyspark.sql.functions import round
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from os import listdir
import pyspark.sql.functions as F
from pyspark.sql.functions import when, col, regexp_replace
from pyspark.sql.types import StringType, DoubleType, IntegerType, TimestampType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


def ExtractDateTransform(df, params, transformationData={}):
    transform_params = params
    dfReturn = df
    feature = transform_params['feature']
    dfReturn = dfReturn.fillna({feature: ''})
    dfReturn = dfReturn.withColumn(
        feature+'dayofmonth', dayofmonth(col(feature)))
    dfReturn = dfReturn.withColumn(feature+'month', month(col(feature)))
    dfReturn = dfReturn.withColumn(feature+'year', year(col(feature)))
    return dfReturn


def BinarizerTransform(df, params, transformationData={}):
    dfReturn = df
    transform_params = params
    feature = transform_params['feature']
    outcol = feature + "_binarizer"
    dfReturn = dfReturn.withColumn("feature_cast", dfReturn[feature].cast("double")).drop(feature)\
        .withColumnRenamed("feature_cast", feature)

    dfReturn = dfReturn.fillna({feature: 0.0})
    binarizer = Binarizer(threshold=float(
        transformationData['threshold']), inputCol=feature, outputCol=outcol)
    binarizedDataFrame = binarizer.transform(dfReturn)

    # binarizedDataFrame=binarizedDataFrame.drop(feature).withColumnRenamed(outcol,feature)

    dfReturn = binarizedDataFrame
    dfReturn = dfReturn.withColumn(feature, round(dfReturn[feature], 2))

    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Patient ID', 'transformation_label': 'String Indexer'}], 'feature': 'Patient ID', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '500', 'mean': '', 'stddev': '', 'min': '004fa9f9-89b7-40c7-b3d3-a9851971937d', 'max': 'ff7facdb-6bbf-411c-94f8-5c96899b0865', 'missing': '0', 'distinct': '500'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Patient ID'}, {'feature_label': 'Patient ID', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Patient ID')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Name', 'transformation_label': 'String Indexer'}], 'feature': 'Name', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Abigail Molina', 'max': 'Zachary Wright', 'missing': '0', 'distinct': '497'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Name'}, {'feature_label': 'Name', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Name')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Gender', 'transformation_label': 'String Indexer'}], 'feature': 'Gender', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Female', 'max': 'Male', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Gender'}, {'feature_label': 'Gender', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Gender')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Medical Condition', 'transformation_label': 'String Indexer'}], 'feature': 'Medical Condition', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Allergies', 'max': 'Urinary Tract Infection', 'missing': '0', 'distinct': '29'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Medical Condition'}, {'feature_label': 'Medical Condition', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Medical Condition')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Treatments', 'transformation_label': 'String Indexer'}], 'feature': 'Treatments', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Antibiotics', 'max': 'Ventilation', 'missing': '0', 'distinct': '37'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Treatments'}, {'feature_label': 'Treatments', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Treatments')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': "Doctor's Notes", 'transformation_label': 'String Indexer'}], 'feature': "Doctor's Notes", 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Advised RICE (Rest, Ice, Compression, Elevation) protocol.', 'max': 'Symptoms should improve within a week.', 'missing': '0', 'distinct': '83'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': "Doctor's Notes"}, {'feature_label': "Doctor's Notes", 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Doctor's Notes')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'Admit Date', 'transformation_label': 'Extract Date'}], 'feature': 'Admit Date', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'Admit Date'}, {'feature_label': 'Admit Date', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('Admit Date')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'Discharge Date', 'transformation_label': 'Extract Date'}], 'feature': 'Discharge Date', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'Discharge Date'}, {'feature_label': 'Discharge Date', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('Discharge Date')
        transformationDF = BinarizerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Bill Amount', 'threshold': 10588.218, 'transformation_label': 'Binarizer'}], 'feature': 'Bill Amount', 'type': 'real', 'selected': 'True', 'replaceby': 'mean', 'stats': {
            'count': '500', 'mean': '10588.71', 'stddev': '16256.78', 'min': '102.55', 'max': '99769.22', 'missing': '0'}, 'transformation': [{'transformation': 'Binarizer', 'selectedAsDefault': 1}], 'updatedLabel': 'Bill Amount'}, {'feature_label': 'Bill Amount', 'threshold': 10588.218, 'transformation_label': 'Binarizer'})
        transformationDF = transformationDF.drop('Bill Amount')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'AdmitSeason', 'transformation_label': 'String Indexer'}], 'feature': 'AdmitSeason', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Fall', 'max': 'Winter', 'missing': '0', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'AdmitSeason'}, {'feature_label': 'AdmitSeason', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('AdmitSeason')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from sklearn.model_selection import train_test_split
from tpot import TPOTRegressor
import pyspark


def functionRegression(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = sparkDF.toPandas()
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTRegressor(verbosity=3, generations=10, max_time_mins=5,
                              n_jobs=-1, random_state=25, population_size=15, use_dask=True)
    tpotModel.fit(X_train, y_train)
    display(" Error rate of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run losregressionHooks.ipynb
try:
	#sourcePreExecutionHook()

	hospitalrecordslosfeaturesready = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/hospital_records_LOS_Features_Ready.csv', 'filename': 'hospital_records_LOS_Features_Ready.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'domain': 'http://172.31.59.158', 'port': '40070', 'dirPath': '/FileStore/platform', 'server_url': '/nexusMax/NexusMaxPlatform/uploads/platform/'}")
	#sourcePostExecutionHook(hospitalrecordslosfeaturesready)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run losregressionHooks.ipynb
try:
	#transformationPreExecutionHook()

	losregressionautofe = TransformationMain.run(hospitalrecordslosfeaturesready,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "Patient ID", "transformation_label": "String Indexer"}], "feature": "Patient ID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "004fa9f9-89b7-40c7-b3d3-a9851971937d", "max": "ff7facdb-6bbf-411c-94f8-5c96899b0865", "missing": "0", "distinct": "500"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Patient ID"}, {"transformationsData": [{"feature_label": "Name", "transformation_label": "String Indexer"}], "feature": "Name", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Abigail Molina", "max": "Zachary Wright", "missing": "0", "distinct": "497"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Name"}, {"transformationsData": [{"feature_label": "Gender", "transformation_label": "String Indexer"}], "feature": "Gender", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Female", "max": "Male", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Gender"}, {"transformationsData": [{"feature_label": "Medical Condition", "transformation_label": "String Indexer"}], "feature": "Medical Condition", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Allergies", "max": "Urinary Tract Infection", "missing": "0", "distinct": "29"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Medical Condition"}, {"transformationsData": [{"feature_label": "Treatments", "transformation_label": "String Indexer"}], "feature": "Treatments", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Antibiotics", "max": "Ventilation", "missing": "0", "distinct": "37"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Treatments"}, {"transformationsData": [{"feature_label": "Doctor's Notes", "transformation_label": "String Indexer"}], "feature": "Doctor's Notes", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Advised RICE (Rest, Ice, Compression, Elevation) protocol.", "max": "Symptoms should improve within a week.", "missing": "0", "distinct": "83"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Doctor's Notes"}, {"transformationsData": [{"feature_label": "Admit Date", "transformation_label": "Extract Date"}], "feature": "Admit Date", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "Admit Date"}, {"transformationsData": [{"feature_label": "Discharge Date", "transformation_label": "Extract Date"}], "feature": "Discharge Date", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "Discharge Date"}, {"transformationsData": [{"feature_label": "Bill Amount", "threshold": 10588.218, "transformation_label": "Binarizer"}], "feature": "Bill Amount", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "10588.71", "stddev": "16256.78", "min": "102.55", "max": "99769.22", "missing": "0"}, "transformation": [{"transformation": "Binarizer", "selectedAsDefault": 1}], "updatedLabel": "Bill Amount"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "LengthOfStayDays", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "15.6", "stddev": "8.64", "min": "1", "max": "30", "missing": "0"}, "updatedLabel": "LengthOfStayDays"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Age", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "49.76", "stddev": "28.58", "min": "1", "max": "101", "missing": "0"}, "updatedLabel": "Age"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "AdmitMonth", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "7.22", "stddev": "3.46", "min": "1", "max": "12", "missing": "0"}, "updatedLabel": "AdmitMonth"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "AdmitQuarter", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2.75", "stddev": "1.1", "min": "1", "max": "4", "missing": "0"}, "updatedLabel": "AdmitQuarter"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "AdmitWeekday", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "3.96", "stddev": "1.93", "min": "1", "max": "7", "missing": "0"}, "updatedLabel": "AdmitWeekday"}, {"transformationsData": [{"feature_label": "AdmitSeason", "transformation_label": "String Indexer"}], "feature": "AdmitSeason", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Fall", "max": "Winter", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "AdmitSeason"}]}))

	#transformationPostExecutionHook(losregressionautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run losregressionHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionRegression(losregressionautofe, ["Age", "AdmitMonth", "AdmitQuarter", "AdmitWeekday", "Patient ID_stringindexer", "Name_stringindexer", "Gender_stringindexer", "Medical Condition_stringindexer", "Treatments_stringindexer", "Doctor's Notes_stringindexer", "Admit Date_dayofmonth", "Admit Date_month", "Admit Date_year", "Discharge Date_dayofmonth", "Discharge Date_month", "Discharge Date_year", "Bill Amount_binarizer", "AdmitSeason_stringindexer"], "LengthOfStayDays")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML ['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    df = df[columnNames]
    R2 = np.round(sklearn.metrics.r2_score(y_test, y_predicted), 1)
    Mean_Squared_Error = np.round(sklearn.metrics.mean_squared_error(y_test, y_predicted), 1)
    Mean_Absolute_Error = np.round(sklearn.metrics.mean_absolute_error(y_test, y_predicted), 1)
    display(" R2 score of Prediction on test data    : %s"%R2)
    display(" Mean Squared Error of Prediction on test data    : %s"%Mean_Squared_Error)
    display(" Mean Absolute Error of Prediction on test data   : %s"%Mean_Absolute_Error)
    display(df.head())
except Exception as ex:
    logging.error(ex)

spark.stop()

